# Shubham Sharma
## IIT BOMBAY

This is a transfer learning implimentation of VGGNet. There can be two casses, eighter we put the trained weights in the convolutional layers as constant or we put them as initial weights while training our network. We'll see both the cases



This is a transfer learning implimentation of VGGnet. We'll put the include_input=False as the input size of the images of cifar-10 is (32,32,3) and then we'll two implimentations. We'll add two dense layers of size 128 and then try to train  the dataset first by training only the layers that we have put and then by training the whole dataset and then see for the results. Due to vanishing gradient problem, we'll take a small portion of the network and will train on that. We rae using cifar10 dataset so the results may or may not come satisfactory due the change in the type of input data.

In [0]:
#Importring important functions

import numpy as np
from keras.applications.vgg16 import VGG16
# from keras_applications.resnet import ResNet50
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten,\
Conv2D,MaxPool2D,Dense,Dropout,Flatten,GlobalAveragePooling2D,\
merge, Input
from keras.models import Model
%matplotlib inline
from keras_applications.imagenet_utils import preprocess_input
from keras.utils import to_categorical 

Preprocessing the data

In [22]:
#######################################################################################################################
epochs=100
batch_size=256
num_of_classes=10
#######################################################################################################################
(x_train, y_train),(x_test,y_test)=cifar10.load_data()

#Checking the shape of the data
input_shape=x_train.shape
print(x_train.shape)
print(y_train.shape)

#reshaping the data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Converting the data to categorical form
y_train = to_categorical(y_train, num_of_classes)
y_test = to_categorical(y_test, num_of_classes)



(50000, 32, 32, 3)
(50000, 1)


Using include_top=True

In [23]:
image_input = Input(shape=(32, 32, 3))
model = VGG16(input_tensor=image_input, include_top=False,weights='imagenet',input_shape=(32, 32, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [26]:
# Last_layer=model.get_layer('block5_pool').output
Last_layer=model.get_layer('block2_pool').output 
x=Flatten(name='flatten')(Last_layer)
x=Dense(128, activation='relu', name='fc1')(x)
x= Dropout(0.5)(x)
x=Dense(128, activation='relu', name='fc2')(x)
x= Dropout(0.5)(x)
out = Dense(num_of_classes, activation='softmax', name= 'output')(x)
model1= Model(image_input,out)
model2= Model(image_input,out)
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

We have made two models model1 and model2. **Model1**  will have all the pretrained weights as constant and  **model2** will have the weights of imagenet as trainable. Let's see the difference.

In [0]:
for layer in model1.layers[:7]:
  model1.trainable = False
  


In [34]:
model1.layers[6].trainable

False

When we have **constant or non-trainable** layers:

In [36]:
adm=keras.optimizers.Adam(lr=1e-4)

model1.compile(loss='categorical_crossentropy',optimizer=adm,metrics=['categorical_accuracy'])

history1 = model1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 8s 162us/step - loss: 2.4081 - categorical_accuracy: 0.0973 - val_loss: 2.3415 - val_categorical_accuracy: 0.0852
Epoch 2/10
50000/50000 [==============================] - 7s 139us/step - loss: 2.4077 - categorical_accuracy: 0.0970 - val_loss: 2.3415 - val_categorical_accuracy: 0.0852
Epoch 3/10
50000/50000 [==============================] - 7s 144us/step - loss: 2.4085 - categorical_accuracy: 0.0950 - val_loss: 2.3415 - val_categorical_accuracy: 0.0852
Epoch 4/10
50000/50000 [==============================] - 7s 144us/step - loss: 2.4064 - categorical_accuracy: 0.0951 - val_loss: 2.3415 - val_categorical_accuracy: 0.0852
Epoch 5/10
50000/50000 [==============================] - 7s 143us/step - loss: 2.4069 - categorical_accuracy: 0.0986 - val_loss: 2.3415 - val_categorical_accuracy: 0.0852
Epoch 6/10
50000/50000 [==============================] - 7s 143us/step - loss: 2.4076 - c

When we have **trainable** layers:

In [37]:
adm=keras.optimizers.Adam(lr=1e-3)

model2.compile(loss='categorical_crossentropy',optimizer=adm,metrics=['categorical_accuracy'])

history2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 9s 174us/step - loss: 1.2481 - categorical_accuracy: 0.5597 - val_loss: 0.8493 - val_categorical_accuracy: 0.7116
Epoch 2/50
50000/50000 [==============================] - 8s 153us/step - loss: 0.8932 - categorical_accuracy: 0.6930 - val_loss: 0.8114 - val_categorical_accuracy: 0.7241
Epoch 3/50
50000/50000 [==============================] - 8s 153us/step - loss: 0.8044 - categorical_accuracy: 0.7244 - val_loss: 0.7945 - val_categorical_accuracy: 0.7309
Epoch 4/50
50000/50000 [==============================] - 8s 152us/step - loss: 0.7390 - categorical_accuracy: 0.7470 - val_loss: 0.7982 - val_categorical_accuracy: 0.7305
Epoch 5/50
50000/50000 [==============================] - 8s 152us/step - loss: 0.6947 - categorical_accuracy: 0.7618 - val_loss: 0.7901 - val_categorical_accuracy: 0.7333
Epoch 6/50
50000/50000 [==============================] - 8s 155us/step - loss: 0.6610 - c

There can be few resons of why we atre not getting satisfactory results in model1. It can be the preprocessing of data when VGG was trained on Imagenet. It can also be the chnage in the number of classes as in ImageNet, there are 1000 different classes but here there are only 10 different classes.